In [41]:
import cv2
import numpy as np
import dlib
import csv
from math import hypot

cap = cv2.VideoCapture("C:/Users/lll/Documents/hz/conda/Eye_detection/data/hadaka_sugisakaya01.mp4")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('C:/Users/lll/Documents/hz/shape_predictor_68_face_landmarks.dat')


# 获取最大的人脸
def _largest_face(dets):
    if len(dets) == 1:
        return 0

    face_areas = [ (det.right()-det.left())*(det.bottom()-det.top()) for det in dets]
    #print(np.sort(face_areas))
    largest_area = face_areas[0]
    largest_index = 0
    for index in range(1, len(dets)):
        if face_areas[index] > largest_area :
            largest_index = index
            largest_area = face_areas[index]

    print("largest_face index is {} in {} faces".format(largest_index, len(dets)))

    return largest_index

def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    #hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_lenght / ver_line_lenght
    return ratio

def get_gaze_ratio(eye_points, facial_landmarks):
    left_eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
    cv2.polylines(frame, [left_eye_region], True, (0, 0, 255), 2)

    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [left_eye_region], True, 255, 2)
    cv2.fillPoly(mask, [left_eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)
    cv2.imshow("mask", eye)

    min_x = np.min(left_eye_region[:, 0])
    max_x = np.max(left_eye_region[:, 0])
    min_y = np.min(left_eye_region[:, 1])
    max_y = np.max(left_eye_region[:, 1])

    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 20, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape
    left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0: height, int(width / 2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)

    if left_side_white == 0:
        gaze_ratio = 1
    elif right_side_white == 0:
        gaze_ratio = 5
    else:
        gaze_ratio = left_side_white / right_side_white
    return gaze_ratio


with open('hadaka_sugisakaya01_gaze_ratio.csv', mode='w',newline ='') as csvfile:
    fieldnames = ["ratio"]
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    while True:
        _, frame = cap.read()
        frame = frame[269: 1095, 737: 1900]
        fps = cap.get(cv2.CAP_PROP_FPS)
        print("fps",fps)
        new_frame = np.zeros((500, 500, 3), np.uint8)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        faces = detector(gray)

        if 0 == len( faces ):
                print( "ERROR: found no face" )
                writer.writerow({'ratio': "null"})
        else:

            largest_index = _largest_face(faces)
            x, y = faces[largest_index].left(), faces[largest_index].top()
            x1, y1 = faces[largest_index].right(), faces[largest_index].bottom()
            cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)

            landmarks = predictor(gray, faces[largest_index])
            #print(face_rectangle)

            # Detect blinking
            left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
            right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
            blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2

            if blinking_ratio > 5.7:
                cv2.putText(frame, "BLINKING", (50, 150), font, 7, (255, 0, 0))


            # Gaze detection
            gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2

            writer.writerow({'ratio': gaze_ratio})
            print(gaze_ratio)
            if gaze_ratio <= 0.3:
                cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0, 0, 255), 3)
                new_frame[:] = (0, 0, 255)
            elif 1 < gaze_ratio < 0.8:
                cv2.putText(frame, "CENTER", (50, 100), font, 2, (0, 0, 255), 3)
            else:
                new_frame[:] = (255, 0, 0)
                cv2.putText(frame, "LEFT", (50, 100), font, 2, (0, 0, 255), 3)



        cv2.imshow("Frame", frame)
        cv2.imshow("New frame", new_frame)


        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()



fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8815536848323733
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8210431654676259
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8481905540789205
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8439574583389784
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8492465269602072
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8364005912786401
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.7963713713713714
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8681818181818182
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.8108996070993091
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8118279569892473
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8372885080202154
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8475609756097562
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8327350625289486
fps 29.97002

largest_face index is 1 in 3 faces
0.9080357142857143
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.9612017678055413
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.8588644614764018
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.8470149253731343
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.8888379485298612
fps 29.97002997002997
largest_face index is 2 in 3 faces
0.8996598639455782
fps 29.97002997002997
largest_face index is 2 in 3 faces
0.8527074018877298
fps 29.97002997002997
largest_face index is 0 in 3 faces
0.8521203089012329
fps 29.97002997002997
largest_face index is 0 in 3 faces
0.9035734543391946
fps 29.97002997002997
largest_face index is 2 in 3 faces
0.8973091976516634
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.8656947890818858
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.9111509468652326
fps 29.97002997002997
largest_face index is 1 in 3 faces
0.8616071428571428
fps 29.97002997002997
largest_face

largest_face index is 1 in 2 faces
0.8913461538461538
fps 29.97002997002997
0.7794117647058824
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8630268199233717
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8316326530612245
fps 29.97002997002997
0.7923242467718794
fps 29.97002997002997
0.9372549019607843
fps 29.97002997002997
0.8563025210084034
fps 29.97002997002997
0.842436974789916
fps 29.97002997002997
0.8529411764705883
fps 29.97002997002997
0.76890756302521
fps 29.97002997002997
0.7575091575091575
fps 29.97002997002997
0.7793040293040293
fps 29.97002997002997
0.9225490196078432
fps 29.97002997002997
0.8736111111111111
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fp

0.6891865079365079
fps 29.97002997002997
0.7264378478664193
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
0.7607059571761713
fps 29.97002997002997
0.6513157894736843
fps 29.97002997002997
0.5571171171171171
fps 29.97002997002997
ERROR: found no face
fps 29.97002997002997
0.7103658536585367
fps 29.97002997002997
0.6737260402057036
fps 29.97002997002997
0.6410794602698651
fps 29.97002997002997
0.6617703533026114
fps 29.97002997002997
0.6957943925233645
fps 29.97002997002997
0.7077981651376146
fps 29.97002997002997
0.6309681697612732
fps 29.97002997002997
0.7220374381188119
fps 29.97002997002997
0.6984463276836159
fps 29.97002997002997
0.6985124659543265
fps 29.97002997002997
0.6682170542635659
fps 29.97002997002997
0.6576923076923077
fps 29.97002997002997
0.635752688172043
fps 29.97002997002997
0.7960340966553493
fps 29.97002997002997
0.7689451769604441
fps 29.97002997002997
0.7215835

largest_face index is 1 in 2 faces
0.7191601049868765
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.7250656167979003
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7415834258231595
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7110649184521907
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.6133333333333333
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.5851111610150461
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.58187125748503
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6993094789704959
fps 29.97002997002997
0.8255597014925373
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6630232558139535
fps 29.97002997002997
0.6261166103331723
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.663952854612345
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.5903967797584819
fps 29.97002997002997
0.6603715170278638
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.6739389

largest_face index is 1 in 2 faces
0.6636210353627415
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.739749201149271
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.8011819681143486
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.7392198004539031
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6885758998435054
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.7218418128730524
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6302717275250908
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.7056398657522254
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.667131307499303
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6628580095553478
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.7797017797017797
fps 29.97002997002997
0.833723470661673
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6889338731443995
fps 29.97002997002997
largest_face index is 1 in 2 faces
0.6566102727528

largest_face index is 1 in 2 faces
0.7786991832823835
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.8261689291101055
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7759264196214343
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7578027465667916
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7898245614035088
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.8166896289609957
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.8320639437562771
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.8255436546575787
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7275219298245614
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7569298245614036
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7592368016816233
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.8768852050355229
fps 29.97002997002997
largest_face index is 0 in 2 faces
0.7907878362423817
fps 29.97002997002997
largest_face

AttributeError: 'NoneType' object has no attribute 'shape'